In [1]:
import xgboost
import bentoml

In [2]:
dtrain = xgboost.DMatrix("data/agaricus.txt.train")

param = {"max_depth": 2, "eta": 1, "objective": "binary:logistic"}
num_round = 2
bst = xgboost.train(param, dtrain, num_round)

In [3]:
bentoml.xgboost.save_model("agaricus", bst)

Model(tag="agaricus:rn2r6rfztwlv5525", path="/home/son/bentoml/models/agaricus/rn2r6rfztwlv5525/")

In [4]:
!bentoml models list

 Tag                        Module           Size      Creation Time       
 agaricus:rn2r6rfztwlv5525  bentoml.xgboost  2.79 KiB  2023-03-03 15:29:41 


In [5]:
#test loading the model as BentoML Runner instance
test_runner = bentoml.xgboost.get("agaricus:latest").to_runner()
test_runner.init_local()

test_runner.run([[0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
                  0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
                  0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
                  1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                  1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
                  0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
                  0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
                  1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
                  1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
                  0, 0, 0, 0, 0, 0, 1]]) 

'Runner.init_local' is for debugging and testing only. Make sure to remove it before deploying to production.


array([0.01241208], dtype=float32)

Create a BentoML Service for serving the model

In [6]:
%%writefile agaricus.py
import typing

import bentoml
import xgboost
from bentoml.io import NumpyNdarray, File

if typing.TYPE_CHECKING:
    import numpy as np

agaricus_runner = bentoml.xgboost.get("agaricus:latest").to_runner()

svc = bentoml.Service("agaricu", runners=[agaricus_runner])

@svc.api(input=NumpyNdarray(), output=NumpyNdarray())
def classify(input_data: "np.ndarray") -> "np.ndarray":
    return agaricus_runner.run(input_data)

Writing agaricus.py


In [7]:
!bentoml serve agaricus.py:svc

2023-03-03T15:51:37+0700 [INFO] [cli] Prometheus metrics for HTTP BentoServer from "agaricus.py:svc" can be accessed at http://localhost:3000/metrics.
2023-03-03T15:51:38+0700 [INFO] [cli] Starting development HTTP BentoServer from "agaricus.py:svc" listening on http://0.0.0.0:3000 (Press CTRL+C to quit)
2023-03-03T15:51:59+0700 [INFO] [dev_api_server:agaricu] 127.0.0.1:41616 (scheme=http,method=GET,path=/favicon.ico,type=,length=) (status=404,type=text/plain; charset=utf-8,length=9) 6.407ms (trace=1084b80589427495e6565c53dc365527,span=e04e6d786a422a4b,sampled=0)
2023-03-03T15:52:06+0700 [INFO] [dev_api_server:agaricu] 127.0.0.1:51538 (scheme=http,method=GET,path=/,type=,length=) (status=200,type=text/html; charset=utf-8,length=2859) 0.363ms (trace=b7d478173718f0a4f6bff07c176af6ea,span=cc16de2858360d64,sampled=0)
2023-03-03T15:52:06+0700 [INFO] [dev_api_server:agaricu] 127.0.0.1:51580 (scheme=http,method=GET,path=/static_content/swagger-initializer.js,type=,length=) (status=200,type=ap

In [8]:
!bentoml build

Error: [bentoml-cli] `build` failed: bentofile "bentofile.yaml" not found
